# Saving lives with AI#Extract disease names form given set of 20000 paragraphs#AUTHOR-NIKHIL MUGGANAWAR

In [1]:
# import libaries
import numpy as np
import pandas as pd
import nltk, pprint
import matplotlib.pyplot as plt
import random

import gzip, os, pickle # gzip for reading the gz files, pickle to save/dump trained model 
import _pickle as cPickle

import sklearn
from sklearn.model_selection import GridSearchCV
from sklearn.grid_search import RandomizedSearchCV

# supress warnings
import warnings
warnings.filterwarnings('ignore')

C:\PGDML\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\PGDML\lib\site-packages\sklearn\grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


# IMPORT DATA

In [2]:
df=pd.read_csv("train.csv",nrows=6000)

In [3]:
df.shape

(6000, 5)

In [4]:
df.columns

Index(['id', 'Doc_ID', 'Sent_ID', 'Word', 'tag'], dtype='object')

In [5]:
from sklearn.model_selection import train_test_split
df_train, df_test= train_test_split(df,train_size=0.50, random_state=101)

In [17]:
disease_indicators=[]
for i in df_train['Word']:
    for j in df_train['tag']:
        if j[0]=='B' or j[0]=='I':
            if i.lower() not in disease_indicators:
                disease_indicators.append(i.lower())
    #print (i)

In [18]:
print(disease_indicators)

['participants', 'and', 'general', 'cholinesterase', 'stimulate', 'de', 'hand', 'among', 'inhibition', 'effect', 'increased', 'to', 'statistically', 'the', 'domain-specific', 'this', '.', 'compared', 'brain', 'juvenile', 'management', 'unfolded', 'in', 'enhancing', 'bone', 'results', 'difference', 'greater', 'green', 'mobile', 'pure', 'blood', 'were', '654.0', 'publicidad', '300', 'published', 'vivo', 'development', 'of', 'paralela', 'thyrotrophin', 'exogenous', ',', 'surgery', 'activity', 'naturaleza', '3', 'tissue', 'show', 'preventing', 'exposure', 'study', 'water', 'between', 'affect', ')', 'other', 'lsd', 'intervention', 'ing1', 'genetic', 'lower', 'addition', 'produced', 'they', 'early', 'manchester', 'occasions', 'bienes', 'laser', '126', 'lipopolysaccharide-stimulated', '72', '(', 'patient', 'spontaneous', 'controls', 'pli', 'inmuebles', 'en', 'finally', 'accumulates', 'evaluated', 'include', 'practical', 'suicide', 'lif', 'folds', 'cell', 'we', 'response', 'tomography', 'julia

In [19]:
# extracts features for the word at index i in a sentence 
def npchunk_features(sentence, i, history):
    word, pos = sentence[i]
    
    # the first word has both previous word and previous tag undefined
    if i == 0:
        prevword, prevpos = "<START>", "<START>"
    else:
        prevword, prevpos = sentence[i-1]

    # gazetteer lookup features (see section below)
    #gazetteer = gazetteer_lookup(word)
    if word in disease_indicators:
        return {"pos": pos, "prevpos": prevpos, 'word':word,
           'word_is_disease_indicator': word,
            } 
    else:
        return {"pos": pos, "prevpos": prevpos, 'word':word,
            }
    

In [20]:
train_postag=df_train['Word']
train_label=df_train['tag']
test_postag=df_test['Word']
test_label=df_test['tag']

In [21]:
train_postag=train_postag.str.lower()
test_postag=test_postag.str.lower()

In [22]:
alphabets=['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']

In [23]:
#filter only those strings which start with a character
def removenumbersandspecialcharacters(list):
    new_list=[]    
    for i in list:
        try:
            if i[0] in alphabets:
                new_list.append(i)
                
        except:
            continue
    return new_list         

In [24]:
train_postag_new=removenumbersandspecialcharacters(train_postag)
test_postag_new=removenumbersandspecialcharacters(test_postag)

In [25]:
# POS tagging sentences
# takes in a list of sentences and returns a list of POS-tagged sentences
# in the form (word, tag)
def pos_tag(sent_list):
    pos_tags = []    
    tagged_words = nltk.pos_tag([sent for sent in sent_list if sent])
    pos_tags.append(tagged_words)
    return pos_tags

In [26]:
train_pos = pos_tag(train_postag_new)
test_pos = pos_tag(test_postag_new)

In [27]:
#train_label=list(map(lambda x:x.split('-')[0],train_label))
#test_label=list(map(lambda x:x.split('-')[0],test_label))
train_label=list(map(lambda x:x.split('indications')[0],train_label))
test_label=list(map(lambda x:x.split('indications')[0],test_label))

In [28]:
#train_label=train_label.tolist()
#test_label=test_label.tolist()

In [29]:
print(train_label)

['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'

In [30]:
newtrain=[]
for x in train_pos:
  for y in x:
    for n in y:
        newtrain.append(n.split(" "))
        #print(new, end=' ')

In [31]:
newtest=[]
for x in test_pos:
  for y in x:
    for n in y:
        newtest.append(n.split(" "))
        #print(new, end=' ')

In [32]:
wordtrain=newtrain[::2]
postrain=newtrain[1::2]
wordtest=newtest[::2]
postest=newtest[1::2]

In [33]:
wordtrain = [item for sublist in wordtrain for item in sublist]
postrain = [item for sublist in postrain for item in sublist]
wordtest = [item for sublist in wordtest for item in sublist]
postest = [item for sublist in postest for item in sublist]

In [34]:
print(wordtrain)

['participants', 'and', 'general', 'cholinesterase', 'stimulate', 'de', 'hand', 'among', 'inhibition', 'effect', 'increased', 'to', 'statistically', 'the', 'and', 'and', 'the', 'domain-specific', 'to', 'this', 'the', 'and', 'and', 'compared', 'brain', 'juvenile', 'management', 'unfolded', 'in', 'enhancing', 'bone', 'results', 'difference', 'greater', 'in', 'green', 'mobile', 'pure', 'blood', 'were', 'publicidad', 'published', 'vivo', 'development', 'of', 'of', 'paralela', 'thyrotrophin', 'exogenous', 'surgery', 'activity', 'naturaleza', 'tissue', 'show', 'preventing', 'exposure', 'study', 'water', 'between', 'affect', 'other', 'lsd', 'the', 'intervention', 'ing1', 'genetic', 'lower', 'addition', 'produced', 'in', 'in', 'and', 'they', 'early', 'manchester', 'occasions', 'were', 'bienes', 'laser', 'and', 'lipopolysaccharide-stimulated', 'compared', 'and', 'patient', 'to', 'spontaneous', 'the', 'controls', 'pli', 'inmuebles', 'en', 'the', 'finally', 'accumulates', 'evaluated', 'include', 

In [35]:
resulttrain=list(zip(wordtrain,postrain,train_label))
resulttest=list(zip(wordtest,postest,test_label))

In [36]:
output = []
for x in train_label:
    if x not in output:
        output.append(x)
print (output)

['O', 'B-', 'I-']


In [37]:
from nltk.corpus import conll2000
from nltk import conlltags2tree, tree2conlltags

In [38]:
train_trees = [conlltags2tree(resulttrain)]
test_trees = [conlltags2tree(resulttest)]

In [39]:
# unigram chunker

from nltk import ChunkParserI

class UnigramChunker(ChunkParserI):    
    def __init__(self, train_sents):
        # convert train sents from tree format to tags
        train_data = [[(t, c) for w, t, c in nltk.chunk.tree2conlltags(sent)] 
                      for sent in train_sents]
        self.tagger = nltk.UnigramTagger(train_data)
        
    def parse(self, sentence):
        pos_tags = [pos for (word, pos) in sentence]
        tagged_pos_tags = self.tagger.tag(pos_tags)
        chunktags = [chunktag for (pos, chunktag) in tagged_pos_tags]
        
        # convert to tree again
        conlltags = [(word, pos, chunktag) for ((word, pos), chunktag) in zip(sentence, chunktags)]
        return nltk.chunk.conlltags2tree(conlltags)
        

In [40]:
# unigram chunker 
unigram_chunker = UnigramChunker(train_trees)
print(unigram_chunker.evaluate(test_trees))

ChunkParse score:
    IOB Accuracy:  98.7%%
    Precision:      0.0%%
    Recall:         0.0%%
    F-Measure:      0.0%%


In [41]:
# extract the list of pos tags
postags = sorted(set([pos for sent in train_trees for (word, pos) in sent.leaves()]))

# for each tag, assign the most likely IOB label
print(unigram_chunker.tagger.tag(postags))

[('CC', 'O'), ('CD', 'O'), ('DT', 'O'), ('EX', 'O'), ('FW', 'O'), ('IN', 'O'), ('JJ', 'O'), ('JJR', 'O'), ('JJS', 'O'), ('MD', 'O'), ('NN', 'O'), ('NNP', 'B-'), ('NNS', 'O'), ('PDT', 'O'), ('PRP', 'O'), ('PRP$', 'O'), ('RB', 'O'), ('RBR', 'O'), ('TO', 'O'), ('VB', 'O'), ('VBD', 'O'), ('VBG', 'O'), ('VBN', 'O'), ('VBP', 'O'), ('VBZ', 'O'), ('WDT', 'O'), ('WP', 'O'), ('WP$', 'O'), ('WRB', 'O')]


In [42]:
# bigram tagger

class BigramChunker(ChunkParserI):    
    def __init__(self, train_sents):
        # convert train sents from tree format to tags
        train_data = [[(t, c) for w, t, c in nltk.chunk.tree2conlltags(sent)] 
                      for sent in train_sents]
        self.tagger = nltk.BigramTagger(train_data)
        
    def parse(self, sentence):
        pos_tags = [pos for (word, pos) in sentence]
        tagged_pos_tags = self.tagger.tag(pos_tags)
        chunktags = [chunktag for (pos, chunktag) in tagged_pos_tags]
        
        # convert to tree again
        conlltags = [(word, pos, chunktag) for ((word, pos), chunktag) in zip(sentence, chunktags)]
        return nltk.chunk.conlltags2tree(conlltags)

In [43]:
# biigram chunker 
bigram_chunker = BigramChunker(train_trees)
print(bigram_chunker.evaluate(test_trees))

ChunkParse score:
    IOB Accuracy:  98.7%%
    Precision:      0.0%%
    Recall:         0.0%%
    F-Measure:      0.0%%


In [48]:
# example sentence
sent_pos = train_pos[0]
sent_pos

[('participants', 'NNS'),
 ('and', 'CC'),
 ('general', 'JJ'),
 ('cholinesterase', 'NN'),
 ('stimulate', 'NN'),
 ('de', 'IN'),
 ('hand', 'NN'),
 ('among', 'IN'),
 ('inhibition', 'JJ'),
 ('effect', 'NN'),
 ('increased', 'VBD'),
 ('to', 'TO'),
 ('statistically', 'RB'),
 ('the', 'DT'),
 ('and', 'CC'),
 ('and', 'CC'),
 ('the', 'DT'),
 ('domain-specific', 'JJ'),
 ('to', 'TO'),
 ('this', 'DT'),
 ('the', 'DT'),
 ('and', 'CC'),
 ('and', 'CC'),
 ('compared', 'VBN'),
 ('brain', 'NN'),
 ('juvenile', 'NN'),
 ('management', 'NN'),
 ('unfolded', 'VBD'),
 ('in', 'IN'),
 ('enhancing', 'VBG'),
 ('bone', 'NN'),
 ('results', 'NNS'),
 ('difference', 'NN'),
 ('greater', 'JJR'),
 ('in', 'IN'),
 ('green', 'JJ'),
 ('mobile', 'JJ'),
 ('pure', 'NN'),
 ('blood', 'NN'),
 ('were', 'VBD'),
 ('publicidad', 'RB'),
 ('published', 'VBN'),
 ('vivo', 'NN'),
 ('development', 'NN'),
 ('of', 'IN'),
 ('of', 'IN'),
 ('paralela', 'NN'),
 ('thyrotrophin', 'VBP'),
 ('exogenous', 'JJ'),
 ('surgery', 'NN'),
 ('activity', 'NN'),
 ('

In [49]:
# features for sentence sent_pos
# each word's features are stored in a dict
for i in range(len(sent_pos)):
    print(npchunk_features(sent_pos, i, history=[]))
    print(' ')

{'pos': 'NNS', 'prevpos': '<START>', 'word': 'participants', 'word_is_disease_indicator': 'participants'}
 
{'pos': 'CC', 'prevpos': 'NNS', 'word': 'and', 'word_is_disease_indicator': 'and'}
 
{'pos': 'JJ', 'prevpos': 'CC', 'word': 'general', 'word_is_disease_indicator': 'general'}
 
{'pos': 'NN', 'prevpos': 'JJ', 'word': 'cholinesterase', 'word_is_disease_indicator': 'cholinesterase'}
 
{'pos': 'NN', 'prevpos': 'NN', 'word': 'stimulate', 'word_is_disease_indicator': 'stimulate'}
 
{'pos': 'IN', 'prevpos': 'NN', 'word': 'de', 'word_is_disease_indicator': 'de'}
 
{'pos': 'NN', 'prevpos': 'IN', 'word': 'hand', 'word_is_disease_indicator': 'hand'}
 
{'pos': 'IN', 'prevpos': 'NN', 'word': 'among', 'word_is_disease_indicator': 'among'}
 
{'pos': 'JJ', 'prevpos': 'IN', 'word': 'inhibition', 'word_is_disease_indicator': 'inhibition'}
 
{'pos': 'NN', 'prevpos': 'JJ', 'word': 'effect', 'word_is_disease_indicator': 'effect'}
 
{'pos': 'VBD', 'prevpos': 'NN', 'word': 'increased', 'word_is_disease

{'pos': 'CC', 'prevpos': 'NN', 'word': 'and', 'word_is_disease_indicator': 'and'}
 
{'pos': 'NN', 'prevpos': 'CC', 'word': 'metropolitana', 'word_is_disease_indicator': 'metropolitana'}
 
{'pos': 'VBN', 'prevpos': 'NN', 'word': 'used', 'word_is_disease_indicator': 'used'}
 
{'pos': 'PRP', 'prevpos': 'VBN', 'word': 'itself', 'word_is_disease_indicator': 'itself'}
 
{'pos': 'VBD', 'prevpos': 'PRP', 'word': 'was', 'word_is_disease_indicator': 'was'}
 
{'pos': 'VBZ', 'prevpos': 'VBD', 'word': 'centres', 'word_is_disease_indicator': 'centres'}
 
{'pos': 'DT', 'prevpos': 'VBZ', 'word': 'the', 'word_is_disease_indicator': 'the'}
 
{'pos': 'DT', 'prevpos': 'DT', 'word': 'the', 'word_is_disease_indicator': 'the'}
 
{'pos': 'NN', 'prevpos': 'DT', 'word': 'cell', 'word_is_disease_indicator': 'cell'}
 
{'pos': 'NN', 'prevpos': 'NN', 'word': 'peak', 'word_is_disease_indicator': 'peak'}
 
{'pos': 'NN', 'prevpos': 'NN', 'word': 'speech', 'word_is_disease_indicator': 'speech'}
 
{'pos': 'NN', 'prevpos

{'pos': 'JJ', 'prevpos': 'NNS', 'word': 'prior', 'word_is_disease_indicator': 'prior'}
 
{'pos': 'NN', 'prevpos': 'JJ', 'word': 'inhibition', 'word_is_disease_indicator': 'inhibition'}
 
{'pos': 'JJ', 'prevpos': 'NN', 'word': 'age-related', 'word_is_disease_indicator': 'age-related'}
 
{'pos': 'IN', 'prevpos': 'JJ', 'word': 'after', 'word_is_disease_indicator': 'after'}
 
{'pos': 'DT', 'prevpos': 'IN', 'word': 'a', 'word_is_disease_indicator': 'a'}
 
{'pos': 'JJ', 'prevpos': 'DT', 'word': 'high', 'word_is_disease_indicator': 'high'}
 
{'pos': 'DT', 'prevpos': 'JJ', 'word': 'the', 'word_is_disease_indicator': 'the'}
 
{'pos': 'JJ', 'prevpos': 'DT', 'word': 'genetic', 'word_is_disease_indicator': 'genetic'}
 
{'pos': 'RB', 'prevpos': 'JJ', 'word': 'maturely', 'word_is_disease_indicator': 'maturely'}
 
{'pos': 'IN', 'prevpos': 'RB', 'word': 'on', 'word_is_disease_indicator': 'on'}
 
{'pos': 'NNS', 'prevpos': 'IN', 'word': 'means', 'word_is_disease_indicator': 'means'}
 
{'pos': 'NNS', 'pr

 
{'pos': 'NN', 'prevpos': 'NN', 'word': 'family', 'word_is_disease_indicator': 'family'}
 
{'pos': 'JJ', 'prevpos': 'NN', 'word': 'pragmatic', 'word_is_disease_indicator': 'pragmatic'}
 
{'pos': 'NNS', 'prevpos': 'JJ', 'word': 'needs', 'word_is_disease_indicator': 'needs'}
 
{'pos': 'RBR', 'prevpos': 'NNS', 'word': 'liver', 'word_is_disease_indicator': 'liver'}
 
{'pos': 'RB', 'prevpos': 'RBR', 'word': 'recently', 'word_is_disease_indicator': 'recently'}
 
{'pos': 'NN', 'prevpos': 'RB', 'word': 'pure', 'word_is_disease_indicator': 'pure'}
 
{'pos': 'IN', 'prevpos': 'NN', 'word': 'of', 'word_is_disease_indicator': 'of'}
 
{'pos': 'DT', 'prevpos': 'IN', 'word': 'that', 'word_is_disease_indicator': 'that'}
 
{'pos': 'NNS', 'prevpos': 'DT', 'word': 'effects', 'word_is_disease_indicator': 'effects'}
 
{'pos': 'RB', 'prevpos': 'NNS', 'word': 'operatively', 'word_is_disease_indicator': 'operatively'}
 
{'pos': 'IN', 'prevpos': 'RB', 'word': 'in', 'word_is_disease_indicator': 'in'}
 
{'pos': 

 
{'pos': 'IN', 'prevpos': 'VBG', 'word': 'in', 'word_is_disease_indicator': 'in'}
 
{'pos': 'NN', 'prevpos': 'IN', 'word': 'age', 'word_is_disease_indicator': 'age'}
 
{'pos': 'IN', 'prevpos': 'NN', 'word': 'as', 'word_is_disease_indicator': 'as'}
 
{'pos': 'IN', 'prevpos': 'IN', 'word': 'of', 'word_is_disease_indicator': 'of'}
 
{'pos': 'NN', 'prevpos': 'IN', 'word': 'imobiliário', 'word_is_disease_indicator': 'imobiliário'}
 
{'pos': 'DT', 'prevpos': 'NN', 'word': 'no', 'word_is_disease_indicator': 'no'}
 
{'pos': 'NN', 'prevpos': 'DT', 'word': 'assay', 'word_is_disease_indicator': 'assay'}
 
{'pos': 'PRP', 'prevpos': 'NN', 'word': 'we', 'word_is_disease_indicator': 'we'}
 
{'pos': 'VB', 'prevpos': 'PRP', 'word': 'be', 'word_is_disease_indicator': 'be'}
 
{'pos': 'JJ', 'prevpos': 'VB', 'word': 'recall', 'word_is_disease_indicator': 'recall'}
 
{'pos': 'NN', 'prevpos': 'JJ', 'word': 'empresas', 'word_is_disease_indicator': 'empresas'}
 
{'pos': 'TO', 'prevpos': 'NN', 'word': 'to', 'w

 
{'pos': 'CC', 'prevpos': 'VBD', 'word': 'and', 'word_is_disease_indicator': 'and'}
 
{'pos': 'NN', 'prevpos': 'CC', 'word': 'debridement', 'word_is_disease_indicator': 'debridement'}
 
{'pos': 'TO', 'prevpos': 'NN', 'word': 'to', 'word_is_disease_indicator': 'to'}
 
{'pos': 'VB', 'prevpos': 'TO', 'word': 'flow', 'word_is_disease_indicator': 'flow'}
 
{'pos': 'DT', 'prevpos': 'VB', 'word': 'the', 'word_is_disease_indicator': 'the'}
 
{'pos': 'NN', 'prevpos': 'DT', 'word': 'comparison', 'word_is_disease_indicator': 'comparison'}
 
{'pos': 'TO', 'prevpos': 'NN', 'word': 'to', 'word_is_disease_indicator': 'to'}
 
{'pos': 'IN', 'prevpos': 'TO', 'word': 'on', 'word_is_disease_indicator': 'on'}
 
{'pos': 'NN', 'prevpos': 'IN', 'word': 'gain', 'word_is_disease_indicator': 'gain'}
 
{'pos': 'NN', 'prevpos': 'NN', 'word': 'vivo', 'word_is_disease_indicator': 'vivo'}
 
{'pos': 'VB', 'prevpos': 'NN', 'word': 'be', 'word_is_disease_indicator': 'be'}
 
{'pos': 'CC', 'prevpos': 'VB', 'word': 'and',

In [50]:
class ConsecutiveNPChunkTagger(nltk.TaggerI): 

    def __init__(self, train_sents):
        train_set = []
        for tagged_sent in train_sents:
            untagged_sent = nltk.tag.untag(tagged_sent)
            history = []
            # compute features for each word
            for i, (word, tag) in enumerate(tagged_sent):
                featureset = npchunk_features(untagged_sent, i, history) 
                train_set.append( (featureset, tag) )
                history.append(tag)
        self.classifier = nltk.NaiveBayesClassifier.train(train_set)

    def tag(self, sentence):
        history = []
        for i, word in enumerate(sentence):
            featureset = npchunk_features(sentence, i, history)
            tag = self.classifier.classify(featureset)
            history.append(tag)
        return zip(sentence, history)

class ConsecutiveNPChunker(nltk.ChunkParserI): 
    def __init__(self, train_sents):
        tagged_sents = [[((w,t),c) for (w,t,c) in
                         nltk.chunk.tree2conlltags(sent)]
                        for sent in train_sents]
        self.tagger = ConsecutiveNPChunkTagger(tagged_sents)

    def parse(self, sentence):
        tagged_sents = self.tagger.tag(sentence)
        conlltags = [(w,t,c) for ((w,t),c) in tagged_sents]
        return nltk.chunk.conlltags2tree(conlltags)

In [51]:
# training the chunker 
chunker = ConsecutiveNPChunker(train_trees)

In [52]:
# evaluate the chunker
print(chunker.evaluate(test_trees))

ChunkParse score:
    IOB Accuracy:  98.3%%
    Precision:      0.0%%
    Recall:         0.0%%
    F-Measure:      0.0%%
